This is the sample model for training the Pandaset dataset to get XYZ and I coordinates to return the type
of objects that are in the dataset : Jacob Igo, Paragon Autonamous


In [49]:
#import libraries
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

In [50]:
#read in data
df = pd.read_csv('SecondSeq_Lidar_FirstPkl.csv')

df.dropna()
df.head()

,x,y,z,i,t,d,class
0,-75.131138,-79.331690,3.511804,7,1.557540e+09,0,5
1,-112.588306,-118.666002,1.423499,31,1.557540e+09,0,5
2,-42.085902,-44.384891,0.593491,7,1.557540e+09,0,14
3,-27.329435,-28.795053,-0.403781,0,1.557540e+09,0,7
4,-6.196208,-6.621082,1.130009,3,1.557540e+09,0,13


In [51]:
#after normalizing
normalized = preprocessing.MinMaxScaler().fit_transform(df)
df2 = pd.DataFrame(normalized)
df2.head()

,0,1,2,3,4,5,6
0,0.201288,0.153526,0.284430,0.027451,0.398635,0.0,0.026316
1,0.098274,0.045645,0.220468,0.121569,0.398659,0.0,0.026316
2,0.292168,0.249373,0.195046,0.027451,0.398657,0.0,0.263158
3,0.332751,0.292131,0.164502,0.000000,0.398653,0.0,0.078947
4,0.390872,0.352946,0.211479,0.011765,0.398634,0.0,0.236842


In [52]:
#split into input/output variables
preX = df.iloc[:, 0:4]
normalX = preprocessing.MinMaxScaler().fit_transform(preX)
X = pd.DataFrame(normalX)
y = df.iloc[:, 6]

In [53]:
#split variables into testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

#initialize model
rf_class = RandomForestClassifier(criterion="gini", min_samples_leaf=1, min_samples_split=10, random_state=1234)

#train
rf_class.fit(X_train, y_train)

#predict
y_pred = rf_class.predict(X_test)
print(y_pred)

[41  5 41 ... 13  7 41]


In [56]:
#get accuracy scores
rf_y_true = y_test

accuracy = metrics.accuracy_score(rf_y_true, y_pred)
precision = metrics.precision_score(rf_y_true, y_pred, average="weighted")
recall = metrics.recall_score(rf_y_true, y_pred, average="weighted")
f1_score = metrics.f1_score(rf_y_true, y_pred, average="weighted")

print("Accuracy:", accuracy)
print("Precision Score:", precision)
print("Recall Score: ", recall)
print("F1 Score: ", f1_score)
print(rf_y_true)

Accuracy: 0.9855289720373368
Precision Score: 0.9845522250362089
Recall Score:  0.9855289720373368
F1 Score:  0.9845579127790323
61528     41
114762     5
22096     41
5427      11
157870     7
          ..
80356     42
160152     7
144969    13
30666      7
63361     41
Name: class, Length: 50031, dtype: int64


C:\Users\jacob\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [58]:
#prediction and ROC / AUC
def plot_roc(rf_y_true, rf_probs, class_labels):
    """Plots ROC curve and displays actual class numbers"""
    rf_y_true_bin = label_binarize(rf_y_true, classes=class_labels)

    for i, class_num in enumerate(class_labels):  # Use actual class numbers
        if np.sum(rf_y_true_bin[:, i]) == 0:  # Skip if no positive samples
            print(f"Skipping class {class_num}: No positive samples in y_true")
            continue
        
        rf_fpr, rf_tpr, _ = roc_curve(rf_y_true_bin[:, i], rf_probs[:, i])
        rf_auc_val = auc(rf_fpr, rf_tpr)
        print(f'Class {class_num}: AUC={rf_auc_val:.2f}')

# Get actual class numbers from y_true
class_labels = np.unique(rf_y_true)  # Get unique class numbers

# Ensure rf_probs has shape (n_samples, n_classes)
rf_probs = rf_class.predict_proba(X_test)  # Get class probabilities
plot_roc(rf_y_true, rf_probs, class_labels)

Class 4: AUC=0.99
Class 5: AUC=1.00
Class 6: AUC=1.00
Class 7: AUC=1.00
Class 8: AUC=0.99
Class 10: AUC=1.00
Class 11: AUC=1.00
Class 13: AUC=1.00
Class 14: AUC=1.00
Class 15: AUC=1.00
Class 18: AUC=1.00
Class 19: AUC=1.00
Class 30: AUC=1.00
Class 31: AUC=1.00
Class 34: AUC=1.00
Class 36: AUC=1.00
Class 37: AUC=1.00
Class 41: AUC=1.00
Class 42: AUC=1.00
